# BasicTauHybridSolver

### Imports

In [1]:
import sys
import os
import numpy
import matplotlib.pyplot as plt
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))
import gillespy2.core.gillespySolver
import gillespy2
from gillespy2.core.gillespyError import SolverError, DirectoryError, BuildError, ExecutionError
from gillespy2.solvers.numpy.basic_tau_hybrid_solver import BasicTauHybridSolver

### Model Instantiation

Model must include rates, species, and reactions

In [2]:
import gillespy2
class MichaelisMenten(gillespy2.Model):
     def __init__(self, parameter_values=None):
            #initialize Model
            gillespy2.Model.__init__(self, name="Michaelis_Menten")
            
            #parameters
            rate1 = gillespy2.Parameter(name='rate1', expression= 0.0017)
            rate2 = gillespy2.Parameter(name='rate2', expression= 0.5)
            rate3 = gillespy2.Parameter(name='rate3', expression = 0.1)
            self.add_parameter([rate1,rate2,rate3])
            
            #Species
            A = gillespy2.Species(name='A', initial_value=301)
            B = gillespy2.Species(name='B', initial_value=120)
            C = gillespy2.Species(name='C', initial_value=0)
            D = gillespy2.Species(name='D', initial_value=0)
            self.add_species([A, B, C, D])
            
            #reactions
            r1 = gillespy2.Reaction(name="r1",reactants={A:1,B:1}, products={C:1},
                   rate=rate1)
            
            r2 = gillespy2.Reaction(name="r2",reactants={C:1}, products={A:1,B:1},
                    rate=rate2)
            
            r3 = gillespy2.Reaction(name="r3",reactants={C:1}, products={B:1,D:1},
                    rate=rate3)
            self.add_reaction([r1,r2,r3])
            self.timespan(numpy.linspace(0,100,101))

### Run Model and plot

In [3]:
model = MichaelisMenten()
for species in model.listOfSpecies.values():
    species.mode='continuous'
%time results = model.run(solver=BasicTauHybridSolver(), tau_tol=.03, switch_tol=.06)

CPU times: user 43.7 ms, sys: 1.03 ms, total: 44.7 ms
Wall time: 40.7 ms


In [4]:
results.plotplotly()

In [5]:
model = MichaelisMenten()
for species in model.listOfSpecies.values():
    species.mode='dynamic'
%time results = model.run(solver=BasicTauHybridSolver, switch_tol=.05)

CPU times: user 459 ms, sys: 13.1 ms, total: 472 ms
Wall time: 433 ms


In [6]:
results.plotplotly()

In [7]:
model = MichaelisMenten()
for species in model.listOfSpecies.values():
    species.mode='discrete'
%prun results = model.run(solver=BasicTauHybridSolver)

In [8]:
results.plotplotly()